# Correos electrónicos SPAM: Un enfoque con Procesamiento de Lenguaje Natural.

## Alumna: Daniela Olivas Mendoza.

Los correos electrónicos no deseados en su bandeja de entrada son molestos ya que perturban la rutina del usuario. Es por eso que las cuentas de correo electrónico ya tiene un filtro spam. Dado que es una de las palicaciones del PLN más utilizadas, vamos a ver como se desarrolló un filtro de spam simple para correos electrónicos.

In [1]:
#Importamos las famosas librerías
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string 
import re

In [2]:
# Insertando los datos
full_corpus = pd.read_csv('SMSSpamCollection.tsv', sep='/t', header= None, names=['label', 'msg_body'])

#Separando los mensajes en 'ham' y 'spam'
ham_text = []
spam_text = []

C:\Users\jorge\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


### Bigrams

Los N-gramos se usan para modelar el lenguaje de función de la predicción de palabras, es decir, predice la siguiente palabra de una oración de palabras N1 anteriores. Bigram es la secuencia de 2 palabras de N-gramos que predice la siguiente palabra de una oración usando la palabra anterior. En lugar de considerar la historia completa de una oración o una secuencia de palabras en particular, un modelo de bigram puede ser ocupado en términos de uuna aproximación de la historia al ocupar uuna historia limitada.

La identificación de un mensaje como 'ham' o 'spam' es una tarea de clasificación ya que la variable de destino tiene valores discretos que son "ham" o "spam". En esta práctica, se usa el modelo bigram, aunque existen muchas técnicas avanzadas que s epueden utilizar para este propósito. Para utilizar el modelo bigram para asignar un mensaje dado como "spam" o "ham", hay varios pasos que deben lograrse:

### 1. Inspección y separación de mensajes en las categorías "ham" y "spam".

nicialmente, el conjunto de datos debe inspeccionarse para ocuparlo y abordarlo para lograr la tarea. El formato de los datos dados, la cantidad de datos proporcionados, la naturaleza de los datos se incluyen en esta inspección para identificar la mejor aproximación posible para la tarea.

El corpus de mensajes dado ha marcado cada mensaje como ham o spam. Además, hay 5568 mensajes en uun DataFrame escrito en inglés que no son objetos nulos. Por lo tanto, el archivo tsv se puede leer usando DataFrame en Python para clasificar esos mensajes de acuerdo con el indicador dado.
            

In [3]:
def separate_msgs():
    for index, column in full_corpus.iterrowa():
        label = column[0]
        message_text = column[1]
        if label == 'ham':
            ham_text.append(message_text)
        elif label == "spam":
            spam_text.append(message_text)

            separate_msgs()

## 2. Procesamiento de texto

El preprocesamiento es la tarea de realizar los pasos de preparación en el corpus de texto sin formato para completar de manera edificente una extracción de texto o procesamiento de lenguaje natural o cualquier tarea que incluya texto sin formato. El preprocesamiento de texto consta de varios pasos, aunque algunos de ellos pueden aplicarse a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto incliye los siguientes pasos de acuerdo con el conjunto de datos:

### Eliminación de signos de puntuación.

In [4]:
#Eliminación de signos de puntuación de los mensajes del correo elec.
def remove_msg_punctuations(email_msg):
    puntuation_removed_msg = "".join([word for word in email_msg if word not in string.punctuation])
    return punctuation_removed_msg

### Convertir minúsculas.

Convertir a minúsculas: la conversión de todos los caracteres del texto en un contexto común, como los soportes en minúsculas, impide identificar dos palabras de manera diferente donde una está en minúscula y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse como iguales, por lo tanto, poner en minúscula todos los caracteres facilita la tarea. Además, las palabras de detención también están em minúsculas, por lo que esto también haría posible eliminar palabras de detención más adelante.

## Tokenizing.

 Tokenizing es la tarea de dividir el texto en partes significativas, es decir, tokens que incluyen oraciones y palabras. Un token se puede considerar como una instancia de una secuencia de caracteres en uun texto particular que se agrupan para proporcionar una unidad semantica util para su posterior procesamiento. En esta tarea, la tokenización de palabras se utiliza cambiando espacios en blanco entre palabras como delimitador. Esto se logra en Python, usando expresiones regulares para dividir una cadena en subcadenas con la función split(), que es un tokenizador básico.

In [5]:
#Convierte el texto en minúsculas y tokenizing de palabras.

def tokenize_into_words(text):
    tokens = re.split('/W+', text)
    return tokens

### Palabras lemantizantes:

La derivación es el proceso de eliminar afijos (sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raíz de palabra. Aunque la lematización está relacionada con la derivación, difiere ya que la lemantización puede capturar formas canónicas basadas en el lema de una palabra. La lematización ocupa un vocabulario y un análisis morfológico de las palabras que lo hacen más rápido y preciso que la derivación. WordNetLemmatizer ha lograo la lematización en lenguaje Python.

In [6]:
#Lemmatizing
word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return " ".join(lemmatized_text)

def preprocessing_msg(corpus):
    categorized_text = pd.DataFrame(corpus)
    categorized_text['non_punc_message_body'] = categorized_text[0].apply(lambda msg: remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body'] = categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_word'] = categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return  categorized_text['lemmatized_msg_word']
    
    


## 3. Extracción de características.

Después de la etapa de preprocesamiento, las características deben extraerse del texto. Las características son las unidades la tarea de clasificación, y las bigrams son las caracteristicas en esta tarea de clasificacion de mensajes. Los bigrams o las caracteristicas se extraen de la tarea de clasificación de mensajes. Los bigrams o las características se extraen del texto preprocesado. Inicialmente, los unigramas se adquieren, y luego esos unigramas se utilizan para obtener los unigramas en cada corpus ("ham" y "spam").

In [7]:
#Extracción de características. Ejemplo: n-grams
def feature_extraction(preprocessed_text):
    bigrams = []
    unigrams_list =[]
    for msg in preprocessed_text:
        #agregando end of and start of al mensaje
        msg = '<s>' +msg +'</s'
        unigrams_list.append(msg.split())
        unigrams = [uni_lists for sub_list in unigrams_list for uni_list in sub_list]
        bigrams.extend(nltk.bigrams(unigrams))
        return bigrams

## 4. Eliminación de Stop Words.

Hay ciertas palabras en un idioma (se utiliza inglés en la practica) que son necesarias para una oracion o una secuencia de palabras, aunque no contribuyen al significado de una frase considerada. La biblioteca del Kit de herramientas de lenguaje natural (NLTK) en Python proporciona palabras de detención comunes en algunos idiomas.


En lugar de eliminar las palabras de detención en el paso de preprocesamiento, se realiza después de extraer las caracteristicas del corpus para evitar la ausencia de bigrams con palabras de una parada (('use', 'your'), ('to', 'win')) al adquirir las funciones, ya que tienen un impacto en el resultado final de la aplicación. Las palabras de detención se pueden ignorar en esta recuperaciónde información orientada a palabras clave debido a su efecto en la precisión de la recuperación.

In [8]:
#Eliminando bigrams solo con stop words
stopwords = nltk.corpus.stopwords.words('english')
def filter_stopwords_bigrams(bigram_list):
    filtered_bigrams = []
    for bigram in bigram_list:
        if bigram[0] in stopwords and bigrams[1] in stopwords:
            continue
            filtered_bigrams.append(bigram)
            return filtered_bigrams

## 5. Obtener distribución de frecuencia de características.

La distribución de frecuencia se utiliza para obtener la frecuencia de aparición de cada elemento de vocabulario en un texto determinado.

In [9]:
# Adquiriendo la frecuencia de características

def ham_bigram_feature_frequency():
    # Frecuencia de características para mensajes ham
    ham_bigram = feature_extraction(preprocessing_msgs(ham_text))
    ham_bigram_frequency = nltk.FreqDist(filter_stopwords_bigrams(ham_bigrams))
    return ham_bigram_frequency

def spam_bigram_feature_frequency():
    # Frecuencia de caracteristicas para mensajes spam
    spam_bigram = feature_extraction(preprocessing_msgs(spam_text))
    spam_bigram_frequency = nltk.FreqDist(filter_stopwords_bigrams(spam_bigrams))
    return spam_bigram_frequency


## 6. Construyendo un modelo para predicción.

El modelo para clasificar dado como 'ham' o 'spam'se ha abordado calculando las probabilidades de bigram dentro de cada corpus. Inicialmente, el mensaje dado debe procesarse previamente para avanzar con la clasificacion, incluida la eliminacion de signos de puntuacion, el cambio de todos los caracteres a minusculas, la tokenizacion y la lematizacion. Luego, los bigrams se extraen del texto preprocesado para calcular finalmente la probabilidad de que el texto esté en cada corpus "ham" o 'spam".

In [10]:
# Calculando probabilidades del bigram
def bigram_probability(message):
    probability_h = 1
    probability_s = 1
    # Preprocesando los mensaje de entrada
    punc_removed_message = "".join(word for word in message if word not in string.punctuation)
    punc_removed_message = '<s> ' +punc_removed_message +' </s>'
    tokenized_msg = re.split('\s+', punc_removed_message)
    lemmatized_msg = [word_lemmatizer.lemmatize(word)for word in tokenized_msg]
    # bigrams para el mensaje
    bigrams_for_msg = list(nltk.bigrams(lemmatized_msg))
    # Eliminamos stop words
    ham_unigrams = [word for word in feature_extraction(preprocessing_msgs(ham_text)) if word not in stopwords]
    spam_unigrams = [word for word in feature_extraction(preprocessing_msgs(spam_text)) if word not in stopwords]
    # Frecuencias de bigrams extraidas
    ham_frequency = ham_bigram_feature_frequency()
    spam_frequency  = spam_bigram_feature_frequency()
    print('========================== Calculando Probabilidades ==========================')
    print('----------- Frecuencias Ham ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        ham_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        ham_probability_of_bigram = ham_frequency[bigram] + 1
        print(bigram, ' ocurre ', ham_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(ham_unigrams):
            ham_probability_denominator += 1
            if(first_unigram == bigram[0]):
                ham_probability_denominator += ham_frequency[first_unigram, second_unigram]
        probability = ham_probability_of_bigram / ham_probability_denominator
        probability_h *= probability
    print('\n')
    print('----------- Frecuencias Spam ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        spam_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        spam_probability_of_bigram = spam_frequency[bigram] + 1
        print(bigram, ' ocurre ', spam_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(spam_unigrams):
            spam_probability_denominator += 1
            if(first_unigram == bigram[0]):
                spam_probability_denominator += spam_frequency[first_unigram, second_unigram]
        probability = spam_probability_of_bigram / spam_probability_denominator
        probability_s *= probability
    print('\n')
    print('Probabilidad Ham: ' +str(probability_h))
    print('Probabildiad Spam: ' +str(probability_s))
    print('\n')
    if(probability_h >= probability_s):
        print('\"' +message +'\" es un mensaje Ham')
    else:
        print('\"' +message +'\" es un mensaje Spam')
    print('\n')
bigram_probability('Click here,  ..to win an iphone 11 pro max')
bigram_probability('Win a brand new car ')

NameError: name 'preprocessing_msgs' is not defined